In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import itertools
import time
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import r2_score

In [2]:
train05 = pd.read_csv('data/2005_data.csv')
train10 = pd.read_csv('data/2010_data.csv')
train15 = pd.read_csv('data/2015_data.csv')
test = pd.read_csv('data/Other_test.csv')
project_data = pd.read_csv('data/Project_Data.csv')

In [8]:
train05

,Country,Alcohol Consumption,GDP,Christian,Muslim,Irreligion,Hindu,Buddhist,Folk religion,Other religion,...,Homicides,"Drug deaths per 100,000",Median_age,Population,conflict_deaths,deaths_to_pop_rate,elec_acc_pct_pop,Urban population,Rural population,pct_female
0,Afghanistan,0.00,345.93,0.01,99.7,0.0,0,0.0,0.0,0.06,...,293.968504,1.59,14.2,2.441120e+07,2114.0,8.659961e-05,25.390894,5824290,19829984,48.391899
1,Albania,7.65,2675.51,18.00,80.3,1.4,0,0.0,0.0,0.20,...,158.000000,0.78,29.1,3.032636e+06,10.0,3.297461e-06,100.000000,1407298,1604189,49.722366
2,Algeria,0.81,3752.10,1.00,98.0,1.8,0,0.0,0.2,0.00,...,488.000000,1.27,23.1,3.295669e+07,324.0,9.831084e-06,98.889610,21159466,11990254,49.423321
3,Andorra,12.55,39782.93,89.50,0.8,8.8,0.50%,0.0,0.0,0.10,...,293.968504,0.31,36.8,7.984500e+04,0.0,0.000000e+00,100.000000,71209,7662,NaN
4,Angola,4.89,3220.08,90.50,1.0,5.1,0,0.0,4.2,0.00,...,293.968504,0.36,15.9,1.945096e+07,11.0,5.655247e-07,29.527786,10882818,8550786,50.397930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,Vietnam,2.63,1273.35,8.20,0.2,29.6,0.21,16.4,45.3,0.40,...,293.968504,0.72,25.4,8.314210e+07,6.0,7.216561e-08,96.099998,22870389,60962273,50.459015
198,World,5.43,8655.98,0.00,0.0,0.0,0,0.0,0.0,0.00,...,293.968504,1.30,26.3,6.558176e+09,59555.0,9.081031e-06,80.647987,3199824687,3310160129,49.612801
199,Yemen,0.34,2536.19,0.20,99.1,0.1,0.6,0.0,0.0,0.00,...,988.000000,0.88,15.2,2.132068e+07,12.0,5.628339e-07,54.904819,5818282,14289134,49.598286
200,Zambia,3.90,918.31,97.60,1.0,0.5,0.1,0.0,0.3,0.90,...,878.000000,0.49,15.0,1.156487e+07,0.0,0.000000e+00,23.321308,4376258,7479986,50.566765
